In [2]:
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 139 kB 4.8 MB/s 


In [3]:
# mnist veri seti üzerinden eğitilmiş 4 katmanlı bir sinir ağının parametrelerini genetic algoritma ile optimize etme
# Genetic algoritma ile optimize edilen parametreler ile sinir ağının test veri seti üzerindeki başarımı

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Genetic algoritma için gerekli kütüphaneler
from deap import base, creator, tools, algorithms

# Veri setini yükleme
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Veri setini normalize etme
x_train = x_train / 255.0
x_test = x_test / 255.0

# Veri setini 1 boyutlu hale getirme
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# Sinir ağını oluşturma
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(784,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Sinir ağını derleme
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Eğitim için gerekli parametreler
batch_size = 128
epochs = 5

# Eğitim
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

# Genetic algoritma için gerekli fonksiyonlar
def evaluate(individual):
    model = Sequential()
    model.add(Dense(int(individual[0]), activation='relu', input_shape=(784,)))
    model.add(Dense(int(individual[1]), activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0, validation_data=(x_test, y_test))
    return history.history['val_accuracy'][-1],

def mutate(individual):
    for i in range(len(individual)):
        if np.random.random() < 0.2:
            individual[i] = np.random.randint(1, 256)
    return individual,

def crossover(ind1, ind2):
    for i in range(len(ind1)):
        if np.random.random() < 0.5:
            ind1[i], ind2[i] = ind2[i], ind1[i]
    return ind1, ind2

# Genetic algoritma için gerekli parametreler
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_int", np.random.randint, 1, 256)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", crossover)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

# Genetic algoritma
pop = toolbox.population(n=10)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)

# Genetic algoritma ile optimize edilen parametreler ile sinir ağının test veri seti üzerindeki başarımı
model = Sequential()
model.add(Dense(int(hof[0][0]), activation='relu', input_shape=(784,)))
model.add(Dense(int(hof[0][1]), activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

# Genetic algoritma ile optimize edilen parametreler ile sinir ağının test veri seti üzerindeki başarımı
print("Genetic algoritma ile optimize edilen parametreler ile sinir ağının test veri seti üzerindeki başarımı: ", history.history['val_accuracy'][-1])

# Genetic algoritma ile optimize edilen parametreler ile sinir ağının eğitim veri seti üzerindeki başarımı
print("Genetic algoritma ile optimize edilen parametreler ile sinir ağının eğitim veri seti üzerindeki başarımı: ", history.history['accuracy'][-1])


11490434/11490434 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/5
469/469 [==============================] - 7s 13ms/step - loss: 0.3119 - accuracy: 0.9108 - val_loss: 0.1471 - val_accuracy: 0.9555
Epoch 2/5
469/469 [==============================] - 4s 9ms/step - loss: 0.1247 - accuracy: 0.9621 - val_loss: 0.1033 - val_accuracy: 0.9690
Epoch 3/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0867 - accuracy: 0.9737 - val_loss: 0.0960 - val_accuracy: 0.9710
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 0.0650 - accuracy: 0.9798 - val_loss: 0.0842 - val_accuracy: 0.9742
Epoch 5/5
469/469 [==============================] - 3s 7ms/step - loss: 0.0501 - accuracy: 0.9841 - val_loss: 0.0831 - val_accuracy: 0.9740
gen	nevals	avg    	std      	min   	max   
0  	10    	0.96771	0.0152096	0.9301	0.9811
1  	6     	0.97616	0.00299606	0.9689	0.9811
2  	10    	0.9764 	0.00148121	0.9742	0.9796
3  	4     	0.97628	0.00161232	0.9718	0.978 
4  	2     	0.92143	0.167978  	0.4175	0.9782
5  	8     	0.97775	0.00213975	0.